# All-in-one Image Processing Web (Streamlit + Colab)

Notebook này tạo **web xử lý ảnh** cho 8 tuần:

- Week01: digital imaging fundamentals  
- Week02: point processing  
- Week03: histogram processing  
- Week04: spatial filtering  
- Week05: frequency domain  
- Week06: PCA & compression  
- Week07: restoration & morphology  
- Week08: segmentation & JPEG-like

Các bước sử dụng:

1. Chạy từng cell từ trên xuống (hoặc **Run all**).
2. Lấy link web từ ngrok (cell 4).
3. Giữ cell cuối (Streamlit) đang chạy trong khi dùng web.

In [ ]:
!pip install -q streamlit opencv-python-headless scikit-image scikit-learn pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 25.2 MB/s eta 0:00:00


## Bước 2 — Nhập ngrok token

Vào [dashboard ngrok](https://dashboard.ngrok.com/get-started/your-authtoken) → copy **authtoken** → dán vào biến dưới.

In [ ]:
from pyngrok import ngrok

NGROK_AUTH_TOKEN = "35iphLCtrywMBsKK94ktKM7iqnz_4MkbZHdAt8eW3bfiASu5k"

if NGROK_AUTH_TOKEN.startswith("PASTE_"):
    print("Chưa thay token.")
else:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    print("Ngrok token set xong.")

✅ Ngrok token set xong.


## Bước 3 — Tạo file `app.py` chứa toàn bộ code web xử lý ảnh

Cell dưới sẽ ghi file `app.py` trong môi trường Colab.

In [ ]:
%%writefile app.py
import io
import numpy as np
import cv2
import streamlit as st
from sklearn.decomposition import PCA
from skimage import morphology, util

# ================== Helper ==================

def read_image(file) -> np.ndarray:
    """Read uploaded image -> RGB uint8"""
    bytes_data = file.read()
    img_array = np.frombuffer(bytes_data, np.uint8)
    bgr = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    return rgb

def to_gray(img_rgb: np.ndarray) -> np.ndarray:
    if len(img_rgb.shape) == 2:
        return img_rgb
    return cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)

def normalize_to_uint8(img: np.ndarray) -> np.ndarray:
    img = np.nan_to_num(img)
    img_min, img_max = img.min(), img.max()
    if img_max - img_min < 1e-5:
        return np.zeros_like(img, dtype=np.uint8)
    img_norm = (img - img_min) / (img_max - img_min)
    return (img_norm * 255).astype(np.uint8)

# ================== Các thuật toán ==================

# ---- Week 01: fundamentals ----

def w1_to_grayscale(img):
    return to_gray(img)

def w1_resize(img, scale):
    h, w = img.shape[:2]
    new_size = (int(w * scale), int(h * scale))
    return cv2.resize(img, new_size, interpolation=cv2.INTER_AREA)

def w1_rotate(img, angle):
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((w/2, h/2), angle, 1.0)
    return cv2.warpAffine(img, M, (w, h))

# ---- Week 02: point processing ----

def w2_negative(img):
    return 255 - img

def w2_log_transform(img, c=1.0):
    img_gray = to_gray(img)
    img_norm = img_gray / 255.0
    log_img = c * np.log(1 + img_norm)
    return normalize_to_uint8(log_img)

def w2_gamma_correction(img, gamma=1.0):
    img_norm = img / 255.0
    corrected = np.power(img_norm, gamma)
    return (corrected * 255).astype(np.uint8)

def w2_contrast_stretch(img, m1=50, m2=200):
    img_gray = to_gray(img).astype(np.float32)
    out = np.zeros_like(img_gray)

    img_gray = np.clip(img_gray, 0, 255)

    # piecewise linear
    out[img_gray < m1] = (img_gray[img_gray < m1] / m1) * 50
    mid_mask = (img_gray >= m1) & (img_gray <= m2)
    out[mid_mask] = 50 + (img_gray[mid_mask] - m1) * (205 / (m2 - m1))
    out[img_gray > m2] = 255
    return out.astype(np.uint8)

# ---- Week 03: histogram processing ----

def w3_hist_equalization(img):
    gray = to_gray(img)
    eq = cv2.equalizeHist(gray)
    return eq

def w3_clahe(img, clip=2.0, tile=8):
    gray = to_gray(img)
    clahe = cv2.createCLAHE(clipLimit=clip, tileGridSize=(tile, tile))
    cl = clahe.apply(gray)
    return cl

# ---- Week 04: spatial filtering ----

def w4_mean_filter(img, k=3):
    return cv2.blur(img, (k, k))

def w4_gaussian_filter(img, k=5, sigma=1.0):
    return cv2.GaussianBlur(img, (k, k), sigma)

def w4_median_filter(img, k=3):
    return cv2.medianBlur(img, k)

def w4_sharpen(img):
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])
    return cv2.filter2D(img, -1, kernel)

def w4_sobel_edge(img):
    gray = to_gray(img)
    gx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    gy = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    mag = np.sqrt(gx**2 + gy**2)
    return normalize_to_uint8(mag)

# ---- Week 05: frequency domain ----

def w5_ideal_lowpass(img, cutoff=30):
    gray = to_gray(img)
    f = np.fft.fft2(gray)
    fshift = np.fft.fftshift(f)

    rows, cols = gray.shape
    crow, ccol = rows // 2, cols // 2
    mask = np.zeros_like(gray, dtype=np.float32)
    Y, X = np.ogrid[:rows, :cols]
    dist = np.sqrt((Y - crow)**2 + (X - ccol)**2)
    mask[dist <= cutoff] = 1

    fshift_filtered = fshift * mask
    f_ishift = np.fft.ifftshift(fshift_filtered)
    img_back = np.fft.ifft2(f_ishift)
    img_back = np.abs(img_back)
    return normalize_to_uint8(img_back)

def w5_ideal_highpass(img, cutoff=30):
    gray = to_gray(img)
    f = np.fft.fft2(gray)
    fshift = np.fft.fftshift(f)

    rows, cols = gray.shape
    crow, ccol = rows // 2, cols // 2
    mask = np.ones_like(gray, dtype=np.float32)
    Y, X = np.ogrid[:rows, :cols]
    dist = np.sqrt((Y - crow)**2 + (X - ccol)**2)
    mask[dist <= cutoff] = 0

    fshift_filtered = fshift * mask
    f_ishift = np.fft.ifftshift(fshift_filtered)
    img_back = np.fft.ifft2(f_ishift)
    img_back = np.abs(img_back)
    return normalize_to_uint8(img_back)

# ---- Week 06: PCA + compression ----

def w6_pca_compress(img, n_components=50):
    """Simple PCA trên ảnh grayscale."""
    gray = to_gray(img).astype(np.float32) / 255.0
    h, w = gray.shape
    # PCA trên chiều rộng
    from sklearn.decomposition import PCA
    pca = PCA(n_components=min(n_components, w))
    transformed = pca.fit_transform(gray)
    reconstructed = pca.inverse_transform(transformed)
    rec = normalize_to_uint8(reconstructed)
    return rec

# ---- Week 07: restoration + morphology ----

def w7_add_gaussian_noise(img, mean=0, sigma=20):
    noisy = img.astype(np.float32) + np.random.normal(mean, sigma, img.shape)
    noisy = np.clip(noisy, 0, 255)
    return noisy.astype(np.uint8)

def w7_median_denoise(img, k=3):
    return cv2.medianBlur(img, k)

def w7_morphology_operation(img, op="erode", k=3):
    gray = to_gray(img)
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)
    kernel = np.ones((k, k), np.uint8)
    if op == "erode":
        result = cv2.erode(binary, kernel, iterations=1)
    elif op == "dilate":
        result = cv2.dilate(binary, kernel, iterations=1)
    elif op == "open":
        result = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
    elif op == "close":
        result = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    else:
        result = binary
    return result

# ---- Week 08: segmentation + "JPEG" style ----

def w8_global_threshold(img, thresh=128):
    gray = to_gray(img)
    _, binary = cv2.threshold(gray, thresh, 255, cv2.THRESH_BINARY)
    return binary

def w8_otsu_threshold(img):
    gray = to_gray(img)
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return binary

def w8_kmeans_segmentation(img, k=3):
    Z = img.reshape((-1, 3))
    Z = np.float32(Z)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    K = k
    ret, label, center = cv2.kmeans(Z, K, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    center = np.uint8(center)
    res = center[label.flatten()]
    return res.reshape((img.shape))

def w8_block_dct_compress(img, q=20):
    """Đơn giản hoá kiểu JPEG: DCT theo block 8x8 trên grayscale."""
    gray = to_gray(img).astype(np.float32) - 128
    h, w = gray.shape
    h8 = h - h % 8
    w8 = w - w % 8
    gray = gray[:h8, :w8]
    out = np.zeros_like(gray)

    Q = np.ones((8, 8), np.float32) * q

    for i in range(0, h8, 8):
        for j in range(0, w8, 8):
            block = gray[i:i+8, j:j+8]
            dct = cv2.dct(block)
            dct_q = np.round(dct / Q)
            block_rec = cv2.idct(dct_q * Q)
            out[i:i+8, j:j+8] = block_rec

    out = out + 128
    return normalize_to_uint8(out)

# ================== UI Streamlit ==================

st.set_page_config(page_title="Image Processing Toolkit", layout="wide")
st.title("Image Processing Toolkit (8 tuần)")

st.sidebar.header("Chọn ảnh")
uploaded_file = st.sidebar.file_uploader("Upload image (JPG/PNG)", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    img = read_image(uploaded_file)
    st.subheader("Ảnh gốc")
    st.image(img, use_column_width=True)
else:
    st.info("Upload 1 ảnh để bắt đầu.")
    st.stop()

st.sidebar.header("Chọn tuần & thuật toán")

week = st.sidebar.selectbox(
    "Chọn tuần",
    [
        "Week01 - Digital imaging fundamentals",
        "Week02 - Point processing",
        "Week03 - Histogram processing",
        "Week04 - Spatial filtering",
        "Week05 - Frequency domain",
        "Week06 - PCA & compression",
        "Week07 - Restoration & morphology",
        "Week08 - Segmentation & JPEG"
    ]
)

result = None  # processed image
extra_info = ""

# ---------------- Week 1 ----------------
if week.startswith("Week01"):
    algo = st.sidebar.selectbox(
        "Thuật toán",
        ["Grayscale", "Resize (scale)", "Rotate"]
    )

    if algo == "Grayscale":
        result = w1_to_grayscale(img)

    elif algo == "Resize (scale)":
        scale = st.sidebar.slider("Scale", 0.1, 2.0, 0.5, 0.1)
        result = w1_resize(img, scale)
        extra_info = f"Scale = {scale}"

    elif algo == "Rotate":
        angle = st.sidebar.slider("Góc quay (độ)", -180, 180, 45, 5)
        result = w1_rotate(img, angle)
        extra_info = f"Angle = {angle}°"

# ---------------- Week 2 ----------------
elif week.startswith("Week02"):
    algo = st.sidebar.selectbox(
        "Thuật toán",
        ["Negative", "Log transform", "Gamma correction", "Contrast stretching"]
    )

    if algo == "Negative":
        result = w2_negative(img)

    elif algo == "Log transform":
        c = st.sidebar.slider("c", 0.5, 5.0, 1.0, 0.1)
        result = w2_log_transform(img, c=c)
        extra_info = f"c = {c}"

    elif algo == "Gamma correction":
        gamma = st.sidebar.slider("gamma", 0.1, 3.0, 1.2, 0.1)
        result = w2_gamma_correction(img, gamma)
        extra_info = f"gamma = {gamma}"

    elif algo == "Contrast stretching":
        m1 = st.sidebar.slider("m1", 0, 255, 50, 1)
        m2 = st.sidebar.slider("m2", 0, 255, 200, 1)
        if m2 <= m1:
            st.warning("m2 phải > m1")
        else:
            result = w2_contrast_stretch(img, m1, m2)
            extra_info = f"m1={m1}, m2={m2}"

# ---------------- Week 3 ----------------
elif week.startswith("Week03"):
    algo = st.sidebar.selectbox(
        "Thuật toán",
        ["Histogram equalization", "CLAHE"]
    )

    if algo == "Histogram equalization":
        result = w3_hist_equalization(img)

    elif algo == "CLAHE":
        clip = st.sidebar.slider("clip limit", 1.0, 10.0, 2.0, 0.5)
        tile = st.sidebar.slider("tile size", 2, 16, 8, 1)
        result = w3_clahe(img, clip, tile)
        extra_info = f"clip={clip}, tile={tile}"

# ---------------- Week 4 ----------------
elif week.startswith("Week04"):
    algo = st.sidebar.selectbox(
        "Thuật toán",
        ["Mean filter", "Gaussian filter", "Median filter", "Sharpen", "Sobel edge"]
    )

    if algo == "Mean filter":
        k = st.sidebar.slider("Kernel size", 3, 15, 3, 2)
        result = w4_mean_filter(img, k)
        extra_info = f"k={k}"

    elif algo == "Gaussian filter":
        k = st.sidebar.slider("Kernel size", 3, 31, 5, 2)
        sigma = st.sidebar.slider("Sigma", 0.1, 10.0, 1.0, 0.1)
        result = w4_gaussian_filter(img, k, sigma)
        extra_info = f"k={k}, sigma={sigma}"

    elif algo == "Median filter":
        k = st.sidebar.slider("Kernel size", 3, 15, 3, 2)
        result = w4_median_filter(img, k)
        extra_info = f"k={k}"

    elif algo == "Sharpen":
        result = w4_sharpen(img)

    elif algo == "Sobel edge":
        result = w4_sobel_edge(img)

# ---------------- Week 5 ----------------
elif week.startswith("Week05"):
    algo = st.sidebar.selectbox(
        "Thuật toán",
        ["Ideal low-pass", "Ideal high-pass"]
    )
    cutoff = st.sidebar.slider("Cutoff frequency", 5, 200, 30, 5)

    if algo == "Ideal low-pass":
        result = w5_ideal_lowpass(img, cutoff)
    elif algo == "Ideal high-pass":
        result = w5_ideal_highpass(img, cutoff)

    extra_info = f"cutoff={cutoff}"

# ---------------- Week 6 ----------------
elif week.startswith("Week06"):
    algo = st.sidebar.selectbox(
        "Thuật toán",
        ["PCA compression (grayscale)"]
    )

    if algo == "PCA compression (grayscale)":
        n_components = st.sidebar.slider("Số thành phần PCA", 5, 200, 50, 5)
        result = w6_pca_compress(img, n_components)
        extra_info = f"n_components={n_components}"

# ---------------- Week 7 ----------------
elif week.startswith("Week07"):
    algo = st.sidebar.selectbox(
        "Thuật toán",
        ["Add Gaussian noise", "Median denoise", "Morphology (erode/dilate/open/close)"]
    )

    if algo == "Add Gaussian noise":
        sigma = st.sidebar.slider("Sigma noise", 1, 80, 20, 1)
        result = w7_add_gaussian_noise(img, sigma=sigma)
        extra_info = f"sigma={sigma}"

    elif algo == "Median denoise":
        k = st.sidebar.slider("Kernel size", 3, 15, 3, 2)
        result = w7_median_denoise(img, k)
        extra_info = f"k={k}"

    elif algo == "Morphology (erode/dilate/open/close)":
        op = st.sidebar.selectbox("Operation", ["erode", "dilate", "open", "close"])
        k = st.sidebar.slider("Kernel size", 3, 15, 3, 2)
        result = w7_morphology_operation(img, op, k)
        extra_info = f"op={op}, k={k}"

# ---------------- Week 8 ----------------
elif week.startswith("Week08"):
    algo = st.sidebar.selectbox(
        "Thuật toán",
        ["Global threshold", "Otsu threshold", "K-means segmentation", "Block DCT (JPEG-like)"]
    )

    if algo == "Global threshold":
        t = st.sidebar.slider("Threshold", 0, 255, 128, 1)
        result = w8_global_threshold(img, t)
        extra_info = f"thresh={t}"

    elif algo == "Otsu threshold":
        result = w8_otsu_threshold(img)

    elif algo == "K-means segmentation":
        k = st.sidebar.slider("K (clusters)", 2, 8, 3, 1)
        result = w8_kmeans_segmentation(img, k)
        extra_info = f"k={k}"

    elif algo == "Block DCT (JPEG-like)":
        q = st.sidebar.slider("Q (quantization strength)", 5, 80, 20, 5)
        result = w8_block_dct_compress(img, q)
        extra_info = f"Q={q}"

# ================== Hiển thị kết quả ==================

st.subheader("Ảnh sau xử lý")
if extra_info:
    st.caption(extra_info)

if result is not None:
    st.image(result, use_column_width=True)
else:
    st.warning("Chưa chọn thuật toán hoặc thuật toán chưa trả ra kết quả.")

Overwriting app.py


## Bước 4 — Mở ngrok tunnel cho cổng 8501

Chạy cell này để lấy URL public truy cập web.

In [ ]:
from pyngrok import ngrok

public_url = ngrok.connect(8501)
print("Public URL:", public_url)

🌐 Public URL: NgrokTunnel: "https://hydrologically-unsubsiding-natosha.ngrok-free.dev" -> "http://localhost:8501"


## Bước 5 — Chạy Streamlit

Giữ cell này **đang chạy** trong khi mở web ở URL ở trên.

In [ ]:
!streamlit run app.py --server.port 8501 --server.headless true --server.enableCORS false

2025-11-20 02:05:42.646 
'server.enableXsrfProtection=true'.
As a result, 'server.enableCORS' is being overridden to 'true'.

More information:
In order to protect against CSRF attacks, we send a cookie with each request.
To do so, we must specify allowable origins, which places a restriction on
cross-origin resource sharing.

If cross origin resource sharing is required, please disable server.enableXsrfProtection.
            



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.6.17.95:8501

2025-11-20 02:08:35.281 The `use_column_width` parameter has been deprecated and will be removed in a future release. Please utilize the `use_container_width` parameter instead.
2025-11-20 02:08:35.380 The `use_column_width` parameter has been deprecated and will be removed in a future release. Please utilize the `use_container_width` parameter instead.
2025-11-20 02:08:49.432 The `use_column